<a href="https://colab.research.google.com/github/mirnanoukari/FoR/blob/main/Assignment1/Alnoukari_Mirna_HA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fundamentals of Robotics**

#### *Assignment 1, Kinematics.*

##### Work of: Mirna Alnoukari


## **1. Robot Description**
Antropomorphic Arm with Spherical Wrist, the robot have 6 joints and is a 6 DoF manipulator.

The robot's configuration for Euler angles arrangement of $zyz$ is described with the figure below:

<img src= "https://drive.google.com/uc?export=view&id=1XGEpzhJmhIcuua4N10MsDC72bl6pZeJW" alt="inverse" width="500" />

 ***Figure 1: Robot's scheme in some configuration***


<img src= "https://drive.google.com/uc?export=view&id=1xbNO042WhLrAKgMld9aCz951Zhn5DJBz" alt="inverse" width="500" />

***Figure 2: zero configuration scheme***




## **2.   Forward Kinematics**

We can obtain the solution for forward kinematics by using translation and rotation matrices directly from Figure 2 for zero configuration scheme.

$$
T = T_{base}\cdot R_z^{q_1}\space T_z^{l_1} \cdot R_x^{q_2}\space T_z^{l_2} \cdot R_x^{q_3}\space T_z^{l_3} \cdot R_z^{q_4} \cdot R_y^{q_5} \cdot R_z^{q_6} \cdot T_{tool}
$$
Note that we will assume no translations for the spherical wrist to simplify our solution.
The same formula can be changed by swapping between consecutive translations and rotations of the same axis. 


Observing the last 3 rotation matrices we get a spherical wrist with $R_{zyz}$

## **3. Inverse Kinematics**

Since we have a spherical wrist we can solve our inverse kinematics problem with Pieper solution. 

We can easily find $q_1$ as the following equation
$
q_1 = \arctan2 (x, y)
$


<img class="center" src="https://drive.google.com/uc?export=view&id=1Uu-bYXzW-l0NY1gowC5tvxmpl_gsbwPx" alt="inverse" width="500" />

***Figure 3: side view of the manipulator***

\\

We need to find $q_2$ and $q_3$, from figure 3, we can find $q_3$ easily as it is equal to $q_3 = π - ϕ$  
We can get angle $ϕ$ using cosine theorem
$$
\cos(ϕ) = \frac{l_2 ^2 + l_3 ^2 - (z - l_1)^2 - (x^2 + y^2)}{2\space l_2 \space l_3 }
$$
Thus, knowing that $\cos (π - ϕ) = -\cos ϕ$

$$
q_3 = \arccos(- \frac{l_2 ^2 + l_3 ^2 - (z - l_1)^2 - (x^2 + y^2)}{2\space l_2 \space l_3 })
$$

\\

We can also observe in figure 3 that $ q_2 = π / 2 - (\theta_1 + \theta_2)$

All the parameters needed can be found:
$$
d = \sqrt(x^2 + y^2)\\
e = z - l_1\\
h = \sqrt(d^2 + e^2)\\
L = \sqrt(x^2 + y^2 + z^2)
$$
$$
\theta_1 = \arccos(\frac{l_1^2 + h^2 - l_2^2} {2 l_1  h^2})\\
\theta_2 = \arccos(\frac{h^2 + l_1^2 - L^2}{2 h l_1})
$$

\\

Now we need to find $q_4, q_5, q_6$ we can split our FK as follow:
$$
T = T_{base}\cdot  T_z^{l_1} \cdot T_{123} \cdot T_{456} \cdot T_z^{l_6} \cdot T_{tool}
$$

$$
T_{123} = T_z^{l_1}\space R_z^{q_1} \cdot T_z^{l_2}\space R_x^{q_2} \cdot R_x^{q_3}\space T_z^{l_3}\space
$$

$$
T_{456} = R_z^{q_4}\cdot R_y^{q_5} \cdot R_z^{q_6}\space
$$
$T_{123}$ is for first 3 joints for position, and $T_{456}$ is for the last 3 joints for orientation.

We can obtain the position of the end effector by solving it analytically.

$$ 
T_{456} = \begin{bmatrix}
 \cos \left(q_4\right) \cos \left(q_5\right) \cos \left(q_6\right)-\sin \left(q_4\right) \sin \left(q_6\right) & \sin \left(q_4\right) \left(-\cos \left(q_6\right)\right)-\sin \left(q_6\right) \cos \left(q_4\right) \cos \left(q_5\right) & \sin \left(q_5\right) \cos \left(q_4\right) & 0 \\
 \sin \left(q_4\right) \cos \left(q_5\right) \cos \left(q_6\right)+\sin \left(q_6\right) \cos \left(q_4\right) & \cos \left(q_4\right) \cos \left(q_6\right)-\sin \left(q_4\right) \sin \left(q_6\right) \cos \left(q_5\right) & \sin \left(q_4\right) \sin \left(q_5\right) & 0 \\
 \sin \left(q_5\right) \left(-\cos \left(q_6\right)\right) & \sin \left(q_5\right) \sin \left(q_6\right) & \cos \left(q_5\right) & 0 \\
 0 & 0 & 0 & 1 \\
\end{bmatrix}
$$

We can solve the previous matrix to find $q_4, q_5, q_6$.

For each angle there are two possible solutions we can obtain, we computed one with its formula and added the negative solution (in case of arccos) and $\pi + ans $ (in case of arctan). In this case, we will have 128 configurations, only 8 will give the right result.
Note that some results are duplicated as the function gives the solution and solution + $2π$.

In [ ]:
import numpy as np
from numpy.linalg import inv
import sympy as sp

def Rx(q):
   return np.array([[1, 0, 0, 0],
              [0, np.cos(q), - np.sin(q), 0],
              [0, np.sin(q), np.cos(q), 0],
              [0, 0, 0, 1]])
   

def Ry (q):
  return np.array([[np.cos(q), 0, np.sin(q), 0],
              [0, 1, 0, 0],
              [- np.sin(q), 0, np.cos(q), 0],
              [0, 0, 0, 1]])

def Rz(q):
  return np.array([[np.cos(q), - np.sin(q), 0, 0],
              [np.sin(q), np.cos(q), 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

def Tx(d):
   return np.array([[1, 0, 0, d],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
   
def Ty(d):
  return np.array([[1, 0, 0, 0],
              [0, 1, 0, d],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

def Tz(d):
  return np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, d],
              [0, 0, 0, 1]])

l = [1, 1, 1, 0, 0, 0]

def T1(q1):
  return Rz(q1) @ Tz(l[0])

def T2(q2):
  return Rx(q2) @ Tz(l[1])

def T3(q3):
  return Rx(q3) @ Tz(l[2])
  
def T4(q4):
  return Rz(q4)

def T5(q5):
  return Ry(q5)

def T6(q6):
  return Rz(q6)

def FK_solve(q, flag):
  A = [T1(q[0]), T2(q[1]), T3(q[2]), T4(q[3]), T5(q[4]), T6(q[5])]
  
  def f(A):
    ret = np.eye(4)
    for a in A:
      ret = ret @ a
    return ret

  if flag == 'ee': return f(A)
  if flag == 'full': return [f(A[:i+1]) for i in range(6)]

def transform_base(trans, q):
  return trans @ FK_solve(q, 'ee')
import itertools as it
#This is a generator that returns the possible solutions for q 
def IK_solve(base_frame, ee_frame):
  frame = inv(base_frame) @ ee_frame
  x, y, z = frame[0, 3], frame[1, 3], frame[2, 3]
  # solving q1
  q1_sols = [-np.arctan2(x, y)]
  q1_sols.append(q1_sols[0] + np.pi)
  
  for q1 in q1_sols:
    #solving q2
    d = np.sqrt(x**2 + y**2)
    e = z - l[0]
    h = np.sqrt(d**2 + e**2)

    theta_1_sols = [np.arccos((l[1]**2 + h**2 - l[2]**2) / (2* l[1] * h))]
    theta_1_sols.append(-theta_1_sols[0])
    
    theta_2_sols = [np.arccos((l[0] ** 2 + h ** 2 - (x ** 2 + y ** 2 + z ** 2)) / (2 * l[0] * h))]
    theta_2_sols.append(-theta_2_sols[0])
    
    q2_sols = [np.pi - (theta_1 + theta_2) for theta_1, theta_2 in it.product(theta_1_sols, theta_2_sols)]
    
    for q2 in q2_sols:
      #solving q3
      q3_sols = [np.arccos(-(l[1]**2 + l[2]**2 - ((z - l[0])**2 + x**2 + y**2)) / (2 * l[1] * l[2]))]
      q3_sols.append(-q3_sols[0])
      
      for q3 in q3_sols:
        # solving q5
        T123 = T1(q1) @ T2(q2) @ T3(q3)
        T456 = inv(T123) @ frame
        # print(f'T456: {T456}')
        q5_sols = [np.arccos(T456[2, 2])]
        q5_sols.append(-q5_sols[0])
        for q5 in q5_sols:
          # solving q4
          if np.abs(q5) > 1e-3:
            q4_sols = [np.arcsin(T456[1, 2] / np.sin(q5))]
            q4_sols.append(np.pi - q4_sols[0])
          else:
            q4_sols = [0]
          for q4 in q4_sols:
            # solving q6
            if np.abs(q5) > 1e-3:
              q6_sols = [np.arccos(-T456[2, 0] / np.sin(q5))]
              q6_sols.append(-q6_sols[0])
            else:
              q6_sols = [np.arccos(T456[0, 0])]
              q6_sols.append(-q6_sols[0])
            for q6 in q6_sols:
              q = [q1, q2, q3, q4, q5, q6]
              if np.sum(np.abs(transform_base(base_frame, q) - ee_frame)) < 1e-3:
                yield q

for sol in IK_solve(np.eye(4), FK_solve([1, 2, 3, 4, 5, 6], 'ee')):
  print(sol)

[0.9999999999999996, 1.9999999999999984, 2.9999999999999996, 0.8584073464102066, 1.283185307179585, 2.858407346410207]
[0.9999999999999996, 1.9999999999999984, 2.9999999999999996, 3.9999999999999996, -1.283185307179585, -0.2831853071795863]
[0.9999999999999996, 4.999999999999998, -2.9999999999999996, -0.8801767676526544, 1.750166968018656, -0.37706940730480676]
[0.9999999999999996, 4.999999999999998, -2.9999999999999996, 2.2614158859371387, -1.750166968018656, 2.7645232462849867]
[4.141592653589793, 1.2831853071795882, 2.9999999999999996, 2.261415885937139, 1.750166968018656, -0.37706940730480587]
[4.141592653589793, 1.2831853071795882, 2.9999999999999996, -0.8801767676526542, -1.750166968018656, 2.7645232462849876]
[4.141592653589793, 4.283185307179588, -2.9999999999999996, 3.9999999999999996, 1.2831853071795851, 2.858407346410207]
[4.141592653589793, 4.283185307179588, -2.9999999999999996, 0.8584073464102063, -1.2831853071795851, -0.2831853071795863]


In [ ]:
#Validation test
successes = 0
total = 0
for _ in range(1000):
  q = 2 * np.pi * np.random.random(6)
  ee = FK_solve(q, 'ee')
  for iq in IK_solve(np.eye(4), ee):
    total += 1
    iee = FK_solve(iq, 'ee')
    if np.sum(np.abs(ee - iee)) > 1e-3:
      print('ERROR')
      print(q)
      print(ee)
      print(iee)
    else:
      successes += 1

print(f'{successes} out of {total} successes, {successes / total * 100}% accuracy')

8028 out of 8028 successes, 100.0% accuracy
